<a href="https://colab.research.google.com/github/uts-strykers/braintumourdetection/blob/STRYK-8-Model-Testing/Assignment2_BrainTumourDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STRYKERS TEAM: Brain Tumour Detection
Members:
Sotero Jr, Leon, Norah

##1. Project Overview: Brain Tumour Detection
Objective: Briefly describe the problem you're trying to solve.
Data Source: Mention where the data comes from and its main features.
Expected Outcome: Define what success looks like for this project.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/42174/braintumourdetection

/content/drive/MyDrive/42174/braintumourdetection


In [ ]:
!git status

Refresh index: 100% (7/7), done.
On branch STRYK-8-Model-Testing
Your branch is up to date with 'origin/STRYK-8-Model-Testing'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Assignment2-BrainTumourDetection.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


#2. Import Libraries
Import necessary Python libraries (e.g., NumPy, pandas, Matplotlib, scikit-learn).

In [ ]:
import matplotlib as plt
import pandas as pd
import numpy as np

#3. Data Collection
Loading Data: Code to load your dataset from a file or web API.
Initial Data Exploration: Display the first few rows of the dataset, list the features and their types.

#4. Data Preprocessing
Cleaning: Handle missing values, remove duplicates, and deal with outliers.
Feature Engineering: Create new features from the existing data if necessary.
Feature Selection: Select relevant features for the model.
Data Transformation: Normalize or standardize data, encode categorical variables.

#5. Exploratory Data Analysis (EDA)
Statistical Summary: Describe the data statistically.
Visualization: Use plots (histograms, box plots, scatter plots) to visualize data and relationships.

#6. Data Splitting
Split the data into training, validation, and test sets.


#7. Model Selection
Choose appropriate machine learning models for the problem.
Briefly describe each model's choice and its assumptions.


#8. Model Training
Training Models: Train models on the training dataset.
Hyperparameter Tuning: Use techniques like grid search or random search to find the best parameters.


#9. Model Testing

In [ ]:
# filter the predictions with the threshold of 0.4
pred_mask = (pred['scores'] >= .4).cpu()
# filter more by using non-maximum suppression with .6 as the threshold
nms_idx = nms(pred['boxes'], pred['scores'], .6).cpu()
nms_mask = torch.zeros_like(pred_mask)
nms_mask[nms_idx] = 1

mask = pred_mask * nms_mask # apply "AND" gate by multiplying the two

indices = torch.arange(0, mask.size(0), 1)[mask]

final_preds = {
    "boxes": pred['boxes'][indices, :],
    "labels": pred['labels'][indices],
    "scores": pred['scores'][indices]
}


image = (255.0 * (image - image.min()) / (image.max() - image.min())).to(torch.uint8)
pred_labels = [f"label {label}: {score:.3f}" for label, score in zip(final_preds["labels"], final_preds["scores"])]
pred_boxes = final_preds["boxes"].long()
colors = ["white"]*len(pred_labels)

# resize the image and bounding boxes
pred_boxes = F.resize_bounding_boxes(pred_boxes, canvas_size=F.get_size(image), size=(132*5, 139*5))
image = F.resize(image, (132*5, 139*5), antialias=True)

print(image.shape, pred_boxes[1])

image_with_boxes = draw_bounding_boxes(image=image, boxes=pred_boxes[0], labels=pred_labels, colors=colors)

Image.fromarray(image_with_boxes.permute(1,2,0).numpy())

#9. Model Evaluation
Validation Performance: Evaluate model performance on the validation set using appropriate metrics (e.g., accuracy, F1 score, ROC AUC).
Cross-Validation: If necessary, perform cross-validation and discuss the results.


#10. Model Interpretation
Discuss model predictions and what they mean in the context of the problem.
Use interpretability tools or techniques if applicable (e.g., SHAP values).


#11. Model Deployment (Optional)
Discuss how the model could be deployed in a real-world application.
Mention any potential challenges and how they could be addressed.


#12. Conclusions
Summarize the key findings.
Discuss any limitations of the current approach and suggest improvements.


#13. References
Cite any references or resources used throughout the project.


#14. Appendix
Include any additional code or analysis that is supplementary to the main project.
This outline serves as a general guide for structuring your machine learning project in a Jupyter Notebook. Depending on the specifics of your project, some sections may need to be modified, expanded, or omitted.

In [ ]:
!git status

On branch STRYK-8-Model-Testing
Your branch is up to date with 'origin/STRYK-8-Model-Testing'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Assignment2-BrainTumourDetection.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git add Assignment2-BrainTumourDetection.ipynb

In [ ]:
!git branch

* STRYK-8-Model-Testing
  main


In [ ]:
!git config --global user.email "sotero.j.saberon@student.uts.edu.au"
!git config --global user.name "saberonjr"

In [ ]:
!git commit -m "STRYK-8 Added Model Testing"

[STRYK-8-Model-Testing b29d123e] STRYK-8 Added Model Testing
 1 file changed, 1 insertion(+), 298 deletions(-)
 rewrite Assignment2-BrainTumourDetection.ipynb (99%)


In [ ]:
!ls /root/.ssh/id_rsa.pub

ls: cannot access '/root/.ssh/id_rsa.pub': No such file or directory


In [ ]:
!ssh -T git@github.com

Host key verification failed.


In [ ]:
!git push -u origin STRYK-8-Model-Testing

Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.
